In [1]:
import numpy as np
import subprocess
import pickle
import time
from ase.spacegroup import crystal
from ase.build import make_supercell
from ase.io.lammpsdata import write_lammps_data, read_lammps_data
import sys
from tqdm import tqdm
from scipy.constants import physical_constants
kB = physical_constants["Boltzmann constant in eV/K"][0]

In [2]:
# Load saved arrays for testing
T=1073
swaps = np.load("MC_test_traj/1/swap_atoms_all_steps.npy")
rands = np.load("MC_test_traj/1/rands_all_steps.npy")
en_stored = np.load("MC_test_traj/1/Eng_all_steps.npy")
accepts = np.load("MC_test_traj/1/accepts_all_steps.npy")
elems = ["Co", "Ni", "Cr", "Fe", "Mn"]

In [3]:
en_stored.shape, accepts.shape

((501,), (500,))

## First make the test lammps command file
### The random seed should be the same in both the initial and the continuation run

In [4]:
with open("MC_test_traj/1/in_run_1.minim", "r") as fl:
    commandList_run_1 = fl.readlines()

with open("MC_test_traj/1/in.minim", "r") as fl:
    commandList_run_2 = fl.readlines()


assert commandList_run_1 == commandList_run_2

# Let's view the commands for the test command file
for command in commandList_run_1:
    print(command)

units 	 metal

atom_style 	 atomic

atom_modify 	 map array

boundary 	 p p p

atom_modify 	 sort 0 0.0

read_data 	 inp_MC.data

pair_style 	 meam

pair_coeff 	 * * /mnt/WorkPartition/Work/Research/UIUC/MDMC/pot/library.meam Co Ni Cr Fe Mn /mnt/WorkPartition/Work/Research/UIUC/MDMC/pot/params.meam Co Ni Cr Fe Mn

minimize		 1e-5 0.0 1000 10000

variable x equal pe

print "$x" file Eng.txt


In [5]:
# Change the following potential file location line since we're one directory up 
with open("in_test.minim", "w") as fl:
    fl.writelines(commandList_run_1)

## Check the energies of all the supercells

In [6]:
NEqb = 5 # we had 5 thermalization steps

supercells = []
# start with the initial supercell
with open("MC_test_traj/1/superInitial.pkl", "rb") as fl:
    sup0 = pickle.load(fl)

sup_run = sup0.copy() # this will be the running supercell which we'll compare with others

# Compute energy
write_lammps_data("inp_MC.data", sup0, specorder=elems)
cmd = subprocess.Popen("$LMPPATH/lmp -in in_test.minim > out_test.txt", shell=True)
rt = cmd.wait()
assert rt == 0

# Check energy
with open("Eng.txt", "r") as fl:
    en = float(fl.readline().split()[0])
assert en_stored[0] == en 

# First do the swaps upto thermalization
for i in range(NEqb):
    acc = accepts[i]
    if acc == 1:
        # get the swap
        sw = swaps[i]
        site1 = sw[0]
        site2 = sw[1]
        
        # swap occupancies in the running supercell
        tmp = sup_run[site1].symbol
        sup_run[site1].symbol = sup_run[site2].symbol
        sup_run[site2].symbol = tmp
    
    # Do the energy computation
    write_lammps_data("inp_MC.data", sup_run, specorder=elems)
    cmd = subprocess.Popen("$LMPPATH/lmp -in in_test.minim > out_test.txt", shell=True)
    rt = cmd.wait()
    assert rt == 0
    with open("Eng.txt", "r") as fl:
        en = float(fl.readline().split()[0])
    
    # Check energy
    assert en == en_stored[i + 1], "{} {}".format(en, en_stored[i + 1])

with open("MC_test_traj/1/chkpt/supercell_{}.pkl".format(NEqb), "rb") as fl:
    sup1 = pickle.load(fl)
        
assert sup1 == sup_run
supercells.append(sup1)

# Up until here, the NEqb^th supercell (starting from 0) has been reached

# Now go through the rest of the supercells
for i in tqdm(range(NEqb + 1, en_stored.shape[0]), position=0, leave=True, ncols=65):
    # Compute energy
    with open("MC_test_traj/1/chkpt/supercell_{}.pkl".format(i), "rb") as fl:
        sup1 = pickle.load(fl)
    supercells.append(sup1)
    
    # compare occupancies
    # first check the if the move was accepted
    # accepts[i-1] is the decision to go from (i-1)th state to (i)th state
    acc = accepts[i-1]
    if acc == 0:
        assert sup1 == sup_run
    
    else:
        # get the swap
        sw = swaps[i-1]
        site1 = sw[0]
        site2 = sw[1]
        
        # swap occupancies in the running supercell
        tmp = sup_run[site1].symbol
        sup_run[site1].symbol = sup_run[site2].symbol
        sup_run[site2].symbol = tmp
        
        assert sup1 == sup_run, "{}".format(i) # Check that the correct supercell was stored.
    
    # compute energies with the running supercell
    write_lammps_data("inp_MC.data", sup_run, specorder=elems)
    cmd = subprocess.Popen("$LMPPATH/lmp -in in_test.minim > out_test.txt", shell=True)
    rt = cmd.wait()
    assert rt == 0
    with open("Eng.txt", "r") as fl:
        en = float(fl.readline().split()[0])
    
    # Check energy
    assert en == en_stored[i], "{} {}".format(en, en_stored[i])
    
    
print("Energy assertions passed")

100%|██████████████████████████| 495/495 [05:14<00:00,  1.58it/s]

Energy assertions passed


## Match the rejected and accepted moves against the random numbers

In [7]:
rejected = []
accepted = []
for enInd in range(en_stored.shape[0] - 1):
    # If the exact same energy occurs, then move must have been rejected
    # Assuming there will be defnitely some change, even if small when a state is changed 
    if en_stored[enInd + 1] - en_stored[enInd] == 0.0:
        assert accepts[enInd] == 0
        rejected.append(enInd)
    else:
        assert accepts[enInd] == 1
        accepted.append(enInd)

In [8]:
# Now Let's check the swaps that were accepted
for move in tqdm(accepted, position=0, leave=True, ncols=65):
    if move < NEqb:
        # the "move"^th move takes us from the "move"^th to the "move + 1"^th state
        # we only have supercell saved from NEqb onwards - so we can check starting with
        # the one before that.
        continue
        
    sup_temp = supercells[move - NEqb].copy()
    tmp = sup_temp[swaps[move, 0]].symbol
    sup_temp[swaps[move, 0]].symbol = sup_temp[swaps[move, 1]].symbol
    sup_temp[swaps[move, 1]].symbol = tmp
    
    # check supercell
    assert sup_temp == supercells[move - NEqb + 1]
    
    # compute energy
    write_lammps_data("inp_MC.data", sup_temp, specorder=elems)
    cmd = subprocess.Popen("$LMPPATH/lmp -in in_test.minim > out_test.txt", shell=True)
    rt = cmd.wait()

    # Read energy
    with open("Eng.txt", "r") as fl:
        en_temp = float(fl.readline().split()[0])
    
    assert np.allclose(en_temp, en_stored[move + 1])
    de = en_stored[move + 1] - en_stored[move]

    test_num = np.exp(-de/(kB*T))
    rand = rands[move]

    assert rand < test_num

print("Acceptance tests okay.")

100%|██████████████████████████| 301/301 [03:07<00:00,  1.60it/s]

Acceptance tests okay.


In [9]:
# Let's first check a swap that was rejected
for move in tqdm(rejected, position=0, leave=True, ncols=65): # index of initial state of the move
    assert np.allclose(en_stored[move], en_stored[move + 1])
    
    if move < NEqb: # the "move"^th move takes us from the "move"^th to the "move + 1"^th state
        continue # we only have supercell saved from NEqb onwards
        
    sup_temp = supercells[move - NEqb].copy()
    assert sup_temp == supercells[move - NEqb + 1]
    
    tmp = sup_temp[swaps[move, 0]].symbol
    sup_temp[swaps[move, 0]].symbol = sup_temp[swaps[move, 1]].symbol
    sup_temp[swaps[move, 1]].symbol = tmp

    # compute energy
    write_lammps_data("inp_MC.data", sup_temp, specorder=elems)
    cmd = subprocess.Popen("$LMPPATH/lmp -in in_test.minim > out_test.txt", shell=True)
    rt = cmd.wait()
    assert rt == 0

    # Read energy
    with open("Eng.txt", "r") as fl:
        en_temp = float(fl.readline().split()[0])

    # Check what the random number was
    rand = rands[move]
    de = en_temp - en_stored[move]
    test_num = np.exp(-de/(kB*T))
    # Check that rand is greater than relative prob.
    assert rand >= test_num, "{} {}".format(rand, test_num)

print("Rejection checks okay.")

100%|██████████████████████████| 199/199 [02:05<00:00,  1.58it/s]

Rejection checks okay.


# Next, check the lammps coordinates that ASE generates

## First, we write out a lammps data file from the initial supercell and read it back in

In [11]:
with open("MC_test_traj/1/superInitial.pkl", "rb") as fl:
    sup0 = pickle.load(fl)

write_lammps_data("write_lammps_test.data", sup0, specorder=elems)
with open("write_lammps_test.data", "r") as fl:
    lammps_file = fl.readlines()

In [12]:
lammps_file[:20]

['write_lammps_test.data (written by ASE) \n',
 '\n',
 '511 \t atoms \n',
 '5  atom types\n',
 '0.0      20.308106755677645  xlo xhi\n',
 '0.0      17.587336353183218  ylo yhi\n',
 '0.0      16.581499731126051  zlo zhi\n',
 '     10.154053377838821      10.154053377838821      5.8624454510610686  xy xz yz\n',
 '\n',
 '\n',
 'Atoms \n',
 '\n',
 '     1   4      1.2692566722298526     0.73280568138263358       2.072687466390756\n',
 '     2   5      2.5385133444597052      1.4656113627652672      4.1453749327815119\n',
 '     3   4      3.8077700166895578       2.198417044147901      6.2180623991722683\n',
 '     4   3      5.0770266889194096      2.9312227255305334      8.2907498655630238\n',
 '     5   4      6.3462833611492631      3.6640284069131677      10.363437331953781\n',
 '     6   2      7.6155400333791157      4.3968340882958019      12.436124798344537\n',
 '     7   2      8.8847967056089683      5.1296397696784348      14.508812264735292\n',
 '     8   4      1.269256672229

## Next, We are going to check the 9 parameters - (x, y, z)lo and hi, and xy, yz, xz <br> and the atomic coordiantes after transformation

In [13]:
# Check all supercells have identical on-lattice positions and lattices
# This will ensure that ASE writes the same initial lammps data for all.
supLatt = sup0.cell[:]
print(supLatt)
for sup in tqdm(supercells, position=0, leave=True, ncols=65):
    assert np.array_equal(sup.cell[:], supLatt)
    for Idx in range(len(sup)):
        assert np.array_equal(sup[Idx].position, sup0[Idx].position)

[[ 0.   14.36 14.36]
 [14.36  0.   14.36]
 [14.36 14.36  0.  ]]


100%|█████████████████████████| 496/496 [00:04<00:00, 109.86it/s]


In [14]:
# Next let's get the vectors we need ready
Avec = supLatt[:, 0].copy()
Bvec = supLatt[:, 1].copy()
Cvec = supLatt[:, 2].copy()

Aunit = Avec / np.linalg.norm(Avec)
Bunit = Avec / np.linalg.norm(Avec)
Cunit = Avec / np.linalg.norm(Avec)

AcrossB_unit = np.cross(Avec, Bvec) / np.linalg.norm(np.cross(Avec, Bvec))
Aunit_cross_B = np.cross(Aunit, Bvec)

## check (x, y, z)lo and hi

In [15]:
xlo = float(lammps_file[4].split()[0])
xhi = float(lammps_file[4].split()[1])

ylo = float(lammps_file[5].split()[0])
yhi = float(lammps_file[5].split()[1])

zlo = float(lammps_file[6].split()[0])
zhi = float(lammps_file[6].split()[1])

print("xlo, xhi : {}, {}".format(xlo, xhi))
print("ylo, yhi : {}, {}".format(ylo, yhi))
print("zlo, zhi : {}, {}".format(zlo, zhi))

xhi_minus_xlo_comp = np.linalg.norm(Avec)
yhi_minus_ylo_comp = np.linalg.norm(Aunit_cross_B)
zhi_minus_zlo_comp = np.abs(np.dot(Cvec, AcrossB_unit))

assert np.math.isclose(xhi_minus_xlo_comp, xhi - xlo)
assert np.math.isclose(yhi_minus_ylo_comp, yhi - ylo)
assert np.math.isclose(zhi_minus_zlo_comp, zhi - zlo)

xlo, xhi : 0.0, 20.308106755677645
ylo, yhi : 0.0, 17.58733635318322
zlo, zhi : 0.0, 16.58149973112605


## check xy, yz, xz

In [16]:
xy = float(lammps_file[7].split()[0])
xz = float(lammps_file[7].split()[1])
yz = float(lammps_file[7].split()[2])
print("xy, xz, yz : {}, {}, {}".format(xy, xz, yz))

xy_comp = np.dot(Bvec, Aunit)
xz_comp = np.dot(Cvec, Aunit)
yz_comp = np.dot(Cvec, np.cross(AcrossB_unit, Aunit))

assert np.allclose(xy, xy_comp)
assert np.allclose(xz, xz_comp)
assert np.allclose(yz, yz_comp)

xy, xz, yz : 10.154053377838821, 10.154053377838821, 5.862445451061069


## Now we check all the co-ordinates

In [17]:
cellLammps = np.array([[xhi - xlo, 0., 0.], [xy, yhi-ylo, 0.], [xz, yz, zhi-zlo]]).T
V = np.dot(Avec, np.cross(Bvec, Cvec))
print(cellLammps)
print(V)

[[20.30810676 10.15405338 10.15405338]
 [ 0.         17.58733635  5.86244545]
 [ 0.          0.         16.58149973]]
5922.339712


In [18]:
matrix_transf = np.zeros((3,3))
matrix_transf[0, :] = np.cross(Bvec, Cvec)
matrix_transf[1, :] = np.cross(Cvec, Avec)
matrix_transf[2, :] = np.cross(Avec, Bvec)
matrix_transf

array([[-206.2096,  206.2096,  206.2096],
       [ 206.2096, -206.2096,  206.2096],
       [ 206.2096,  206.2096, -206.2096]])

In [19]:
for atom in tqdm(range(len(sup0)), position=0, leave=True, ncols=65):
    cartpos = sup0[atom].position
    symbol = sup0[atom].symbol
    atomID = elems.index(symbol) + 1 # lammps starts with 1
    
    lammps_x = float(lammps_file[12 + atom].split()[2])
    lammps_y = float(lammps_file[12 + atom].split()[3])
    lammps_z = float(lammps_file[12 + atom].split()[4])
    
    cartpos_transf = matrix_transf @ cartpos
    cartpos_transf = cellLammps @ cartpos_transf
    cartpos_transf /= V
    
    assert np.allclose(cartpos_transf, np.array([lammps_x, lammps_y, lammps_z]))
    assert atomID == int(lammps_file[12 + atom].split()[1])

100%|████████████████████████| 511/511 [00:00<00:00, 8085.53it/s]
